![](https://img.shields.io/badge/tutorial2/2-lightgrey)
[![](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamindb/blob/main/docs/tutorial1.ipynb)
[![](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/dMtrt8YMSdl6z8)

# Tutorial: Features & labels

In the previous tutorial ({doc}`/tutorial`), we learned about about how to leverage basic metadata for artifacts to access data (query, search, cache & load).

Here, we walk through annotating & validating data with features & labels to improve:

1. Finding data: Which collections measured expression of cell marker `CD14`? Which characterized cell line `K562`? Which collections have a test & train split? Etc.
2. Using data: Are there typos in feature names? Are there typos in sampled labels? Are units of features consistent? Etc.

:::{hint}

This is a low-level tutorial aimed at a basic understanding of features and labels.

If you want to a high-level flow for validating and annotating features and labels, see {doc}`annotate`.

:::

:::{dropdown} What was LaminDB's most basic inspiration?

The pydata family of data objects is at the heart of most data science, ML & comp bio workflows: `DataFrame`, `AnnData`, `pytorch.DataLoader`, `zarr.Array`, `pyarrow.Table`, `xarray.Collection`, ...

We couldn’t find a tool to link these objects to context so that they could be analyzed in context:

- data sources, data transformations, models, users & pipelines that performed transformations (provenance)
- the domain entities in which data is generated and modeled (features & labels)

:::

In [ ]:
import lamindb as ln
import pandas as pd

ln.settings.verbosity = "hint"

## Register metadata

Features and labels are the primary ways of registering domain-knowledge related metadata in LaminDB.

Features represent measurement dimensions (e.g. organism) and labels represent measured values (e.g. iris setosa, iris versicolor, iris virginica).

In statistics, you'd say a feature is a categorical or numerical variable. A label is a simple category. Categorical variables draw their values from a set of categories.

### Register labels

We study 3 organism of the Iris plant: `setosa`, `versicolor` & `virginica`. Let's create 3 labels with {class}`~lamindb.ULabel`.

In [ ]:
organisms = [ln.ULabel(name=name) for name in ["setosa", "versicolor", "virginica"]]
ln.save(organisms)

{class}`~lamindb.ULabel` enables you to manage an in-house ontology to manage all kinds of generic labels.

:::{dropdown} What are alternatives to ULabel?

In a complex project, you'll likely want dedicated typed registries for selected label types, e.g., {class}`~bionty.Gene`, {class}`~bionty.Tissue`, etc. See: {doc}`/bio-registries`.

{class}`~lamindb.ULabel`, however, will get you quite far and scale to ~1M labels.

:::

Anticipating that we'll have many different labels when working with more data, we'd like to express that all 3 labels are organism labels:

In [ ]:
is_organism = ln.ULabel(name="is_organism").save()
is_organism.children.set(organisms)
is_organism.view_parents(with_children=True)

In addition to organism, we want like to track the studies that produced the data:

In [ ]:
studies = [ln.ULabel(name=name) for name in ["study0", "study1", "study2"]]
ln.save(studies)
is_study = ln.ULabel(name="is_study").save()
is_study.children.set(studies)
is_study.view_parents(with_children=True)

:::{dropdown} Why label a dataset by study?

We can then

1. query all artifacts link to this experiment
2. model it as a confounder when we'll analyze similar data from a follow-up experiment, and concatenate data using the label as a feature in a data matrix

:::

### Register features

For every set of studied labels (measured values), we typically also want an identifier for the corresponding measurement dimension: the feature.

When we integrate datasets, feature names will label columns that store data.

Let's create and save two {class}`~lamindb.Feature` records to identify measurements of the iris organism label and the study:

In [ ]:
ln.Feature(name="iris_organism_name", type="category").save()
ln.Feature(name="study_name", type="category").save()
# create a lookup object so that we can access features with auto-complete
features = ln.Feature.lookup()

## Validate & link labels

We already looked at the metadata for `study0`, before: 

In [ ]:
meta_artifact = ln.Artifact.filter(key="iris_studies/study0_raw_images/meta.csv").one()
meta = meta_artifact.load(index_col=0)  # load a dataframe
meta.head()

### Validate metadata

Depending on the data generation process, such metadata might or might not match the labels we defined in our registries.

Let's validate the labels by mapping the values stored in the artifact on the {class}`~lamindb.ULabel` registry:

In [ ]:
ln.ULabel.validate(meta["1"], field="name")

Everything passed and no fixes are needed!

If validation doesn't pass, {meth}`~lamindb.core.CanValidate.standardize` and {meth}`~lamindb.core.CanValidate.inspect` will help standardize data.

### Label artifacts

You can label an artifact by calling `artifact.labels.add()` and pass a single or multiple labels, and optionally, the corresponding feature.

Let's do this based on the labels in `meta.csv`:

In [ ]:
ln.Artifact.df()

In [ ]:
study_artifacts = ln.Artifact.filter(key__startswith="iris_studies/", suffix="").all()
study_labels = ln.ULabel.filter(name="is_study").one().children.all()
for artifact, study in zip(study_artifacts, study_labels):
    artifact.labels.add(study, feature=features.study_name)
    df = pd.read_csv(artifact.path / "meta.csv", index_col=0)
    organism_labels = ln.ULabel.from_values(df["1"].unique())
    artifact.labels.add(organism_labels, feature=features.iris_organism_name)

### Query artifacts by labels

Using the new annotations, you can now query image artifacts by organism & study labels:

In [ ]:
ulabels = ln.ULabel.lookup()
artifact = ln.Artifact.filter(ulabels=ulabels.study0).first()

We also see them when calling {meth}`~lamindb.core.Data.describe`:

In [ ]:
artifact.describe()

### Label collections

Labeling collections works in the same way as labeling artifacts:

In [ ]:
collection = ln.Collection.filter(name__startswith="Iris collection", version="1").one()
collection.labels.add(ulabels.study0, feature=features.study_name)
all_organism_labels = ln.ULabel.filter(parents__name="is_organism").all()
collection.labels.add(all_organism_labels, feature=features.iris_organism_name)

In [ ]:
collection.describe()

## Run an ML model

Let's now run a mock ML model that transforms the images into 4 high-level features.

In [ ]:
def run_ml_model() -> pd.DataFrame:
    transform = ln.Transform(name="Petal & sepal regressor", type="pipeline")
    ln.track(transform=transform)
    input_data = ln.Collection.filter(name__startswith="Iris collection", version="1").one()
    input_paths = [
        path.download_to(path.name) for path in input_data.artifacts[0].path.glob("*")
    ]
    # apply ML model
    output_data = ln.core.datasets.df_iris_in_meter_study1()
    return output_data


df = run_ml_model()

The output is a dataframe:

In [ ]:
df.head()

And this is the pipeline that produced the dataframe:

In [ ]:
ln.core.run_context.transform.view_parents()

### Register the output data

Let's first register the features of the transformed data:

In [ ]:
new_features = ln.Feature.from_df(df)
ln.save(new_features)

:::{dropdown} How to track units of features?

Use the `unit` field of {class}`~lamindb.Feature`. In the above example, you'd do:

```python
for feature in features:
    if feature.type == "number":
        feature.unit = "m"  # SI unit for meters
        feature.save()
```

:::

We can now validate & register the dataframe in one line:

In [ ]:
artifact = ln.Artifact.from_df(
    df,
    description="Iris study 1 - after measuring sepal & petal metrics",
)
artifact.save()

In [ ]:
artifact.features.add(new_features)

### Feature sets

Get an overview of linked features:

In [ ]:
artifact.features

You'll see that they're always grouped in sets that correspond to records of {class}`~lamindb.FeatureSet`.

:::{dropdown} Why does LaminDB model feature sets, not just features?

1. Performance: Imagine you measure the same panel of 20k transcripts in 1M samples. By modeling the panel as a feature set, you'll only need to store 1M instead of 1M x 20k = 20B links.
2. Interpretation: Model protein panels, gene panels, etc.
3. Data integration: Feature sets provide the currency that determines whether two collections can be easily concatenated.

These reasons do not hold for label sets. Hence, LaminDB does not model label sets.

:::

A `slot` provides a string key to access feature sets. It's typically the accessor within the registered data object, here `pd.DataFrame.columns`.

Let's use it to access all linked features:

In [ ]:
artifact.features["columns"].df()

There is one categorical feature, let's add the organism labels:

In [ ]:
organism_labels = ln.ULabel.filter(parents__name="is_organism").all()
artifact.labels.add(organism_labels, feature=features.iris_organism_name)

Let's now add study labels:

In [ ]:
artifact.labels.add(ulabels.study0, feature=features.study_name)

In addition to the `columns` feature set, we now have an `external` feature set:

In [ ]:
artifact.features

This is the context for our artifact:

In [ ]:
artifact.describe()
artifact.view_lineage()

See the database content:

In [ ]:
ln.view(registries=["Feature", "FeatureSet", "ULabel"])

## Manage follow-up data

Assume that a couple of weeks later, we receive a new dataset in a follow-up study 2.

Let's track a new analysis:

In [ ]:
ln.settings.transform.stem_uid = "dMtrt8YMSdl6"
ln.settings.transform.version = "1"
ln.track()

### Register a joint collection

Assume we already ran all preprocessing including the ML model.

We get a DataFrame and store it as an artifact:

In [ ]:
df = ln.core.datasets.df_iris_in_meter_study2()
ln.Artifact.from_df(df, description="Iris study 2 - transformed").save()

Let's load it:

In [ ]:
artifact2 = ln.Artifact.filter(description="Iris study 2 - transformed").one()

We can now store the joint collection:

In [ ]:
collection = ln.Collection(
    [artifact, artifact2], name="Iris flower study 1 & 2 - transformed"
)
collection.save()

### Auto-concatenate datasets

Because both datasets measured the same validated feature set, we can auto-concatenate the collection:

In [ ]:
collection.load().tail()

We can also access & query the underlying two artifact objects:

In [ ]:
collection.artifacts.df()

Or look at their data lineage:

In [ ]:
collection.view_lineage()

Or look at the database:

In [ ]:
ln.view()

This is it! 😅

If you're interested, please check out guides & use cases or make an issue on GitHub to [discuss](https://github.com/laminlabs/lamindb/issues/new).

## Appendix

### Manage metadata

#### Avoid duplicates

Let's create a label `"project1"`:

In [ ]:
ln.ULabel(name="project1").save()

We already created a `project1` label before, let's see what happens if we try to create it again:

In [ ]:
label = ln.ULabel(name="project1")
label.save()

Instead of creating a new record, LaminDB loads and returns the existing record from the database.

If there is no exact match, LaminDB will warn you upon creating a record about potential duplicates.

Say, we spell "project 1" with a white space:

In [ ]:
ln.ULabel(name="project 1")

To avoid inserting duplicates when creating new records, a search compares whether a similar record already exists.

You can switch it off for performance gains via {attr}`~lamindb.core.Settings.upon_create_search_names`.

#### Update & delete records

In [ ]:
label = ln.ULabel.filter(name="project1").first()
label

In [ ]:
label.name = "project1a"
label.save()
label

In [ ]:
label.delete()

### Manage storage

#### Change default storage

The default storage location is:

In [ ]:
ln.settings.storage

You can change it by setting `ln.settings.storage = "s3://my-bucket"`.

#### See all storage locations

In [ ]:
ln.Storage.df()

In [ ]:
# clean up what we wrote in this notebook
!rm -r lamin-tutorial
!lamin delete --force lamin-tutorial